In [ ]:
import numpy as np
import pandas as pd
import scipy.stats

pd.options.mode.chained_assignment = None
from matplotlib import pyplot as plt, rcParams
# import cv2
import seaborn as sns

sns.set(style="white", context="paper")
from cycler import cycler
import os, sys
import glob
from datetime import datetime, timedelta
from itertools import combinations, product
import base64
from PIL import Image
from io import BytesIO as _BytesIO
import requests
import json
import pickle
from datetime import datetime
from IPython.display import display, Markdown, Latex
from sklearn.metrics import *
import collections
from copy import deepcopy
import traceback
from sympy import Point, Polygon
from decorators import *
from smartprint import smartprint as sprint
from scipy.spatial.distance import cdist
from sklearn.cluster import DBSCAN
import mmcv
from mmtrack.apis import inference_mot, init_model as init_tracking_model
# import plotly
# from pandas_profiling import ProfileReport

pd.options.display.max_columns = None
def printm(s): return display(Markdown(s))
    
SERVER_CACHE_DIR = '/mnt/ci-nas-cache/edulyzeV2/cache_compute_4/fixed_face'
os.makedirs(SERVER_CACHE_DIR,exist_ok=True)

track_analysis_meta_cache = f'{SERVER_CACHE_DIR}/analysis_tracking/meta_info'
base_dir = '/mnt/ci-nas-cache/edulyzeV2/pose_face_gaze_emb_fixed_face/'

track_analysis_session_data = f'{SERVER_CACHE_DIR}/analysis_tracking/session_tracking_info'
os.makedirs(track_analysis_session_data,exist_ok=True)

postprocessed_id_map_data_dir = f'{SERVER_CACHE_DIR}/analysis_tracking/processed_id_maps'
os.makedirs(postprocessed_id_map_data_dir, exist_ok=True)

emb_analysis_session_data = f'{SERVER_CACHE_DIR}/analysis_emb/session_emb_info_new'
os.makedirs(emb_analysis_session_data,exist_ok=True)

embmatched_id_raw_data_dir = f'{SERVER_CACHE_DIR}/analysis_emb/embmatched_id_raw'
os.makedirs(embmatched_id_raw_data_dir,exist_ok=True)

embmatched_id_map_data_dir = f'{SERVER_CACHE_DIR}/analysis_tracking/embmatched_id_maps_new'
os.makedirs(embmatched_id_map_data_dir, exist_ok=True)

cross_session_input_data_dir = f'{SERVER_CACHE_DIR}/analysis_emb/cross_session_input'
os.makedirs(cross_session_input_data_dir, exist_ok=True)

id_viz_cache_root = f'{SERVER_CACHE_DIR}/analysis_emb/session_matching_info'
os.makedirs(id_viz_cache_root, exist_ok=True)

v5_id_start_stop_data_dir = f'{SERVER_CACHE_DIR}/analysis_emb/v5_id_start_stop_info'
os.makedirs(v5_id_start_stop_data_dir, exist_ok=True)


## Get frame file data for all sessions

In [ ]:

frame_file_data = {}
for course_idx, course_dir in enumerate(glob.glob(f"{base_dir}/*")):
    course_name = course_dir.split("/")[-1]
    course_cache_file = f"{track_analysis_meta_cache}/{course_name}"
    if os.path.exists(course_cache_file):
        frame_file_data[course_name] = pickle.load(open(course_cache_file,"rb"))
        continue
    frame_file_data[course_name]={}
        
    for session_idx, session_dir in enumerate(glob.glob(f"{course_dir}/*")):
        session_name = session_dir.split("/")[-1]
        frame_file_data[course_name][session_name] = {}
        frame_files = glob.glob(f"{session_dir}/*")
        frame_file_names = [xr.split("/")[-1] for xr in frame_files]
        if 'end.pb' in frame_file_names:
            frame_file_data[course_name][session_name]['is_completed']=True
        else:
            frame_file_data[course_name][session_name]['is_completed']=False            
        frame_ids = [int(xr.split(".")[0]) for xr in frame_file_names if not (xr=='end.pb')]
        frame_file_data[course_name][session_name]['frame_ids'] = sorted(frame_ids)
        frame_file_data[course_name][session_name]['dir_location'] = session_dir
        print(f"Got metadata for course: {course_idx}-{course_name}, session:{session_idx}-{session_name}")
    pickle.dump(frame_file_data[course_name],open(course_cache_file,"wb")) 
        
frame_file_data.keys()


In [ ]:
session_filter_list = [
 #    'classinsight-cmu_05681A_ghc_4301_201905011630',
 # 'classinsight-cmu_05681A_ghc_4301_201904171630',
 # 'classinsight-cmu_05681A_ghc_4301_201902201630',
 # 'classinsight-cmu_05681A_ghc_4301_201904101630',
 # 'classinsight-cmu_05681A_ghc_4301_201901231630',
                       
 # 'classinsight-cmu_05418A_ghc_4102_201902251200',
 # 'classinsight-cmu_05418A_ghc_4102_201904081200',
 # 'classinsight-cmu_05418A_ghc_4102_201905011200',
 # 'classinsight-cmu_05418A_ghc_4102_201904291200',
 # 'classinsight-cmu_05418A_ghc_4102_201904011200',
                       
 'classinsight-cmu_05748A_ghc_4101_201902141630',
 'classinsight-cmu_05748A_ghc_4101_201904021630',
 'classinsight-cmu_05748A_ghc_4101_201902051630',
 'classinsight-cmu_05748A_ghc_4101_201902281630',
 'classinsight-cmu_05748A_ghc_4101_201903071630',
                       
 # 'classinsight-cmu_21127J_ghc_4102_201904230930',
 # 'classinsight-cmu_21127J_ghc_4102_201903260930',
 # 'classinsight-cmu_21127J_ghc_4102_201904160930',
 # 'classinsight-cmu_21127J_ghc_4102_201904300930',
 # 'classinsight-cmu_21127J_ghc_4102_201903190930',
                       
 # 'classinsight-cmu_05410A_ghc_4301_201904151500',
 # 'classinsight-cmu_05410A_ghc_4301_201902251500',
 # 'classinsight-cmu_05410A_ghc_4301_201904081500',
 # 'classinsight-cmu_05410A_ghc_4301_201904221500',
 # 'classinsight-cmu_05410A_ghc_4301_201902181500',
                       
 'classinsight-cmu_17214B_ph_a21_201902271030',
 'classinsight-cmu_17214B_ph_a21_201903061030',
 'classinsight-cmu_17214B_ph_a21_201904031030',
 'classinsight-cmu_17214B_ph_a21_201904101030',
 'classinsight-cmu_17214B_ph_a21_201904241030',
                       
 'classinsight-cmu_17214C_ph_225b_201903201130',
 'classinsight-cmu_17214C_ph_225b_201904101130',
 'classinsight-cmu_17214C_ph_225b_201904171130',
 'classinsight-cmu_17214C_ph_225b_201904241130',
 'classinsight-cmu_17214C_ph_225b_201905011130',
                       
 # 'classinsight-cmu_05410B_ghc_4211_201902111500',
 # 'classinsight-cmu_05410B_ghc_4211_201903181500',
 # 'classinsight-cmu_05410B_ghc_4211_201904081500',
 # 'classinsight-cmu_05410B_ghc_4211_201904151500',
 # 'classinsight-cmu_05410B_ghc_4211_201904221500',
 # 'classinsight-cmu_05410B_ghc_4211_201901281500'
]


# Get frames data across all sessions

In [ ]:
frame_file_data = {}
for course_idx, course_dir in enumerate(glob.glob(f"{base_dir}/*")):
    course_name = course_dir.split("/")[-1]
    course_cache_file = f"{track_analysis_meta_cache}/{course_name}"
    if os.path.exists(course_cache_file):
        frame_file_data[course_name] = pickle.load(open(course_cache_file,"rb"))
        continue
    frame_file_data[course_name]={}
        
    for session_idx, session_dir in enumerate(glob.glob(f"{course_dir}/*")):
        session_name = session_dir.split("/")[-1]
        frame_file_data[course_name][session_name] = {}
        frame_files = glob.glob(f"{session_dir}/*")
        frame_file_names = [xr.split("/")[-1] for xr in frame_files]
        if 'end.pb' in frame_file_names:
            frame_file_data[course_name][session_name]['is_completed']=True
        else:
            frame_file_data[course_name][session_name]['is_completed']=False            
        frame_ids = [int(xr.split(".")[0]) for xr in frame_file_names if not (xr=='end.pb')]
        frame_file_data[course_name][session_name]['frame_ids'] = sorted(frame_ids)
        frame_file_data[course_name][session_name]['dir_location'] = session_dir
        print(f"Got metadata for course: {course_idx}-{course_name}, session:{session_idx}-{session_name}")
    pickle.dump(frame_file_data[course_name],open(course_cache_file,"wb")) 
        
frame_file_data.keys()


# Get id_start_stop info for all sessions

In [ ]:
# writing a generic loop to get embedding info from all courses in frame file data

for course_idx, course in enumerate(frame_file_data):
    for session_idx, session_id in enumerate(frame_file_data[course]):
        if session_id.split("-front")[0] not in session_filter_list:
            print(f"Session {session_id} not in session filter list, skipping...")
            continue
        v5_id_start_stop_cache_file = f"{v5_id_start_stop_data_dir}/{session_id}.csv"
        try:
            if not os.path.exists(v5_id_start_stop_cache_file):
                session_dir = frame_file_data[course][session_id]['dir_location']
                frame_ids = frame_file_data[course][session_id]['frame_ids']
                
                session_tracking_cache_file = f"{track_analysis_session_data}/{session_id}.pb"
                session_preprocessed_id_map_file = f"{postprocessed_id_map_data_dir}/{session_id}.pb"
                session_eligible_pairs_map_file = f"{embmatched_id_map_data_dir}/{session_id}.csv"

                printm("### get final id mapping for old to new ids with emb processing.")
                old_to_new_id_map = pickle.load(open(session_preprocessed_id_map_file,"rb"))
                df_session_eligible_pairs = pd.read_csv(session_eligible_pairs_map_file)
                
                printm("### Correct new ids with dict from eligible pairs")
                eligible_id_map_dict = {}
                for id_pair in df_session_eligible_pairs.id_pair.values:
                    (id1, id2) = eval(id_pair)
                    print(id1, id2)
                    if id2 in eligible_id_map_dict:
                        eligible_id_map_dict[id1] = eligible_id_map_dict[id2]
                    else:
                        eligible_id_map_dict[id2] = id1
                
                sprint(eligible_id_map_dict)
                
                printm("### correct created eligible map for once more")
                for key in sorted(list(eligible_id_map_dict.keys())):
                    key_value = eligible_id_map_dict[key]
                    if key_value in eligible_id_map_dict.keys():
                        eligible_id_map_dict[key] = eligible_id_map_dict[key_value]
                sprint(eligible_id_map_dict) 
                # sprint({kr:old_to_new_id_map[kr] for kr in old_to_new_id_map if (not old_to_new_id_map[kr]==10000)}) 
                
                sprint("replacing ids in old to new id maps")
                for old_id in old_to_new_id_map:
                    if old_to_new_id_map[old_id] in eligible_id_map_dict:
                        print(f"replacing {old_id}:{old_to_new_id_map[old_id]} -->{eligible_id_map_dict[old_to_new_id_map[old_id]]}")
                        old_to_new_id_map[old_id] = eligible_id_map_dict[old_to_new_id_map[old_id]]
                # sprint({kr:old_to_new_id_map[kr] for kr in old_to_new_id_map if (not old_to_new_id_map[kr]==10000)}) 

                printm("### map new ids to tracking dataframe")
                df_tracking_new = pickle.load(open(session_tracking_cache_file,"rb")).transpose()
                total_idxs = df_tracking_new.index.max()
                for old_id in old_to_new_id_map:
                    new_id = old_to_new_id_map[old_id]
                    if not new_id==10000:
                        new_id_col = f'N{new_id}'
                        if new_id_col not in df_tracking_new:
                            df_tracking_new[new_id_col] = None
                        df_tracking_new[new_id_col] =  df_tracking_new[new_id_col].where(~df_tracking_new[new_id_col].isnull(), df_tracking_new[old_id])
                    df_tracking_new = df_tracking_new.drop(old_id, axis=1)

                printm("### get id start stop dataframe")
                col_start_stop_idxs = []
                for col in df_tracking_new.columns:
                    one_idxs = df_tracking_new.index[np.where(df_tracking_new[col]==1)[0]].values
                    col_start_stop_idxs.append([col, one_idxs.min(), one_idxs.max()])
                df_id_start_stop = pd.DataFrame(col_start_stop_idxs, columns=['id','min_idx','max_idx'])
                df_id_start_stop['total_idxs'] = df_id_start_stop['max_idx']-df_id_start_stop['min_idx']
                df_id_start_stop['id'] = df_id_start_stop['id'].apply(lambda x: int(x[1:]))
                df_id_start_stop.to_csv(v5_id_start_stop_cache_file, index=False)
                printm(f"## Got id start stop info for session: {course_idx}-{course}, session:{session_idx}-{session_id}")
            else:
                ...
                printm(f"### FILE EXISTS: id start stop info for session: {course_idx}-{course}, session:{session_idx}-{session_id}")
        except:
            printm(f"## ERROR: Unable to get id start stop info for: {course_idx}-{course}, session:{session_idx}-{session_id}")
            unfinished_sessions.append((course, session_id))
            print(traceback.format_exc())
    

# Get Groundtruth for course

In [ ]:
course = '17214C'
df_gt = pd.read_csv(f"groundtruth/{course}.tsv",sep='\t',index_col=0)
df_gt


In [ ]:
# get GT pairs


In [ ]:
# convert ground truth into gt_pairs


# New algorithm for session pair id matching based on consistencies of ids present

## Describing the algorithm here.

We are trying to find consistent ids across all sessions, and only do matching for those ids.

- Step 1: filter out ids which are consistent more than 75% of the session from both sessions.
- Step 2: Use 3 tiered rules to match ids across those sessions based on gaze and clu matches.

In [ ]:
# get input for all sessions
course_input_dict = {}
idstartstop_input_dict = {}
sessions = [xr for xr in session_filter_list if (course in xr)]
for session in sessions:
    session_input_file = f'{cross_session_input_data_dir}/{session}-front.pb'
    session_id_start_stop_file = f'{v5_id_start_stop_data_dir}/{session}-front.csv'
    course_input_dict[session] = pickle.load(open(session_input_file,"rb"))
    idstartstop_input_dict[session] = pd.read_csv(session_id_start_stop_file)
course_input_dict.keys()


In [ ]:
df_filtered_gt = deepcopy(df_gt)
for col in df_filtered_gt:
    df_filtered_gt[col] = df_filtered_gt[col].apply(lambda x: list(map(int,str(x).split(","))) if not (x=='-1') else [])
df_filtered_gt


In [ ]:
course_input_dict_raw= deepcopy(course_input_dict)

for session in sessions:
    df_id_start_stop_session = idstartstop_input_dict[session]
    df_id_start_stop_session['presence_fraction'] = df_id_start_stop_session.total_idxs / df_id_start_stop_session.total_idxs.max()
    filtered_ids = df_id_start_stop_session[df_id_start_stop_session.presence_fraction>0.5]['id'].values
    course_input_dict[session] = {xr: course_input_dict[session][xr] for xr in course_input_dict[session] if xr in filtered_ids}
    session_col = f'Session-{session[-8:-4]}'
    df_filtered_gt[session_col] = df_filtered_gt[session_col].apply(lambda x: [xr for xr in x if (xr in filtered_ids)])
    sprint(session, course_input_dict[session].keys())
    # sprint(df_id_start_stop_session)
    # _ = plt.figure(figsize=(20,15))
    # for row_idx, row in df_id_start_stop_session.iterrows():
    #     plt.axhline(y=row_idx, xmin=row['min_idx']/df_id_start_stop_session.total_idxs.max(),xmax=row['max_idx']/df_id_start_stop_session.total_idxs.max())
    # plt.yticks(range(df_id_start_stop_session.shape[0]), range(df_id_start_stop_session.shape[0]))
    # plt.grid() 
    sprint(filtered_ids)
df_filtered_gt    
    

In [ ]:
# get possible gt matches
true_matches = []
session_cols = list(df_filtered_gt.columns)
for idx_colA in range(len(session_cols)):
    for idx_colB in range(idx_colA+1, len(session_cols)):
        match_val_pairs = zip(df_filtered_gt[session_cols[idx_colA]].values.tolist(), df_filtered_gt[session_cols[idx_colB]].values.tolist())
        for ids_colA, ids_colB in match_val_pairs:
            for id_colA in ids_colA:
                for id_colB in ids_colB:
                    session_keyA = session_cols[idx_colA].split("-")[-1]
                    session_keyB = session_cols[idx_colB].split("-")[-1]
                    true_matches.append([session_keyA, session_keyB, f'{session_keyA}_{id_colA}', f'{session_keyB}_{id_colB}'])
df_true_matches = pd.DataFrame(true_matches, columns=['sessionA','sessionB','idA','idB'])
df_true_matches['gt'] = 1
df_true_matches             


In [ ]:
course_input_dict[session].keys()


In [ ]:
type='constrained'


In [ ]:
# matching ids0.40	0.3	50
# original 0.3, 0.2, 40
MATCH_THRESHOLD=0.35
MATCH_PAIR_MAX_THRESHOLD = 0.2
MATCH_PAIR_AREA_THRESHOLD = 20

# unconstrained
if type=='unconstrained':
    MATCH_THRESHOLD=0.35
    MATCH_PAIR_MAX_THRESHOLD = 0.2
    MATCH_PAIR_AREA_THRESHOLD = 20
    print(type, MATCH_THRESHOLD, MATCH_PAIR_MAX_THRESHOLD, MATCH_PAIR_AREA_THRESHOLD)

final_matches = []
for (sessionA, sessionB) in product(sorted(sessions), sorted(sessions)):
    sessionA_key, sessionB_key = sessionA.split("_")[-1][4:8], sessionB.split("_")[-1][4:8]
    if not (sessionA==sessionB):
        # if sessionA not in session_matches:
        # match session A and session B based on gaze clustering
        match_scores_gaze = {}
        match_scores_clu  = {}
        for idA,idB in product(course_input_dict[sessionA].keys(), course_input_dict[sessionB].keys()):

            if not (type=='unconstrained'):
                # check if there exists a gt match for idA in SessionB
                idA_key = f'{sessionA_key}_{idA}'
                idB_key = f'{sessionB_key}_{idB}'
                idA_matches = df_true_matches[(df_true_matches.sessionB==sessionB_key) & (df_true_matches.idA==idA_key)].idB.values
                if len(idA_matches)<=0:
                    idA_matches = df_true_matches[(df_true_matches.sessionA==sessionB_key) & (df_true_matches.idB==idA_key)].idA.values
                    if len(idA_matches)<=0:
                        # print(f"Skipping {idA},{idB} as no matches for {idA_key} in {sessionB_key}")
                        continue
                idB_matches = df_true_matches[(df_true_matches.sessionA==sessionA_key) & (df_true_matches.idB==idB_key)].idA.values
                if len(idB_matches) <= 0:
                    idB_matches = df_true_matches[(df_true_matches.sessionB==sessionA_key) & (df_true_matches.idA==idB_key)].idB.values
                    if len(idB_matches) <= 0:
                        # print(f"Skipping {idA},{idB} as no matches for {idB_key} in {sessionA_key}")
                        continue
                
            
            gaze_embA, gaze_embB = course_input_dict[sessionA][idA]['gaze_emb'], course_input_dict[sessionB][idB]['gaze_emb']                
            clu_embA, clu_embB = course_input_dict[sessionA][idA]['cluster_emb'], course_input_dict[sessionB][idB]['cluster_emb']                
            
            if idA not in match_scores_gaze:
                match_scores_gaze[idA] = {}
            if idA not in match_scores_clu:
                match_scores_clu[idA] = {}
                
            if gaze_embA is None or gaze_embB is None:
                match_scores_gaze[idA][idB] = np.inf
            else:
                match_distance = cdist(gaze_embA.reshape(1,-1), gaze_embB.reshape(1,-1))[0][0]
                match_scores_gaze[idA][idB] = match_distance

            if clu_embA is None or clu_embB is None:
                match_scores_clu[idA][idB] = np.inf
            else:
                match_distance = cdist(clu_embA.reshape(1,-1), clu_embB.reshape(1,-1))[0][0]
                match_scores_clu[idA][idB] = match_distance
        
        df_match_gaze = pd.DataFrame(match_scores_gaze) 
        df_match_clu = pd.DataFrame(match_scores_clu) 
        gaze_cols = df_match_gaze.columns.values.tolist()
        clu_cols = df_match_clu.columns.values.tolist()
        all_cols = np.unique(gaze_cols+clu_cols)
        for col in all_cols:
            if col not in df_match_clu.columns:
                df_match_clu[col] = np.inf
            if col not in df_match_gaze.columns:
                df_match_clu[col] = np.inf
                
            sessionB_matches = deepcopy(df_match_clu[col]).sort_values().index.values.tolist() + \
                                deepcopy(df_match_gaze[col]).sort_values().index.values.tolist()
            if type=='unconstrained':
                sessionB_matches = deepcopy(df_match_clu[col]).sort_values().head(3).index.values.tolist() + \
                                deepcopy(df_match_gaze[col]).sort_values().head(3).index.values.tolist()
            sessionB_matches = np.unique(sessionB_matches)
            for match_id in sessionB_matches:
                if (match_scores_clu[col][match_id]<MATCH_THRESHOLD) | (match_scores_gaze[col][match_id]<MATCH_THRESHOLD):
                    col_face_area = course_input_dict[sessionA][col]['face_width_med'] * course_input_dict[sessionA][col]['face_height_med']
                    match_face_area = course_input_dict[sessionB][match_id]['face_width_med'] * course_input_dict[sessionB][match_id]['face_height_med']
                    rel_diff = np.abs(col_face_area-match_face_area)*100/min(col_face_area,match_face_area)
                    final_matches.append((sessionA_key, sessionB_key, f'{sessionA_key}_{col}', f'{sessionB_key}_{match_id}', match_scores_gaze[col][match_id], match_scores_clu[col][match_id], col_face_area, match_face_area))

df_final_matches = pd.DataFrame(final_matches, columns=['sessionA','sessionB','idA','idB','match_score_gaze','match_score_clu','face_areaA','face_areaB'])
df_final_matches.info()
df_final_matches.head(40)


In [ ]:
final_course_ids = {}
session_pair_matches = {}
session_keys = df_final_matches.sessionA.unique()

for sessionA_idx in range(len(session_keys)):
    for sessionB_idx in range(sessionA_idx+1, len(session_keys)):
        # find all direct pairs
        
        direct_pairs = {}
        s1, s2 = session_keys[sessionA_idx], session_keys[sessionB_idx]
        df_pair_matches = df_final_matches[(df_final_matches.sessionA==s1) & (df_final_matches.sessionB==s2)]
        df_pair_matches = df_pair_matches.sort_values(by=['face_areaA','face_areaB'],ascending=False)
        df_pair_matches['area_diff'] = np.abs(df_pair_matches['face_areaA'] - df_pair_matches['face_areaB'])*100/np.minimum(df_pair_matches['face_areaA'],df_pair_matches['face_areaB'])
        df_pair_matches['avg_match_score'] = (df_pair_matches['match_score_gaze'] + df_pair_matches['match_score_clu'])/2
        # df_pair_matches['avg_match_score'] = df_pair_matches['match_score_gaze']
        # df_pair_matches['avg_match_score'] = df_pair_matches['match_score_clu']
        # get all eligible pairs with direct pairing
        matched_idBs = []
        for idA in df_pair_matches.idA.unique():
            idA_matches = df_pair_matches[(df_pair_matches.idA==idA) & (~(df_pair_matches.idB.isin(matched_idBs)))].sort_values(by='avg_match_score')[['idB','avg_match_score','area_diff']].values
            if idA_matches.shape[0] > 0:
                matched_idB, match_score, area_diff = idA_matches[0][0],idA_matches[0][1],idA_matches[0][2]
                matched_idBs.append(matched_idB)
                direct_pairs[(idA,matched_idB)] = (match_score, area_diff)

        inverse_pairs = {}
        s1, s2 = session_keys[sessionB_idx], session_keys[sessionA_idx]
        df_pair_matches = df_final_matches[(df_final_matches.sessionA==s1) & (df_final_matches.sessionB==s2)]
        df_pair_matches = df_pair_matches.sort_values(by=['face_areaA','face_areaB'],ascending=False)
        df_pair_matches['area_diff'] = np.abs(df_pair_matches['face_areaA'] - df_pair_matches['face_areaB'])*100/np.minimum(df_pair_matches['face_areaA'],df_pair_matches['face_areaB'])
        df_pair_matches['avg_match_score'] = (df_pair_matches['match_score_gaze'] + df_pair_matches['match_score_clu'])/2
        # df_pair_matches['avg_match_score'] = df_pair_matches['match_score_gaze']
        # df_pair_matches['avg_match_score'] = df_pair_matches['match_score_clu']
        # get all eligible pairs with direct pairing
        matched_idBs = []
        for idA in df_pair_matches.idA.unique():
            idA_matches = df_pair_matches[(df_pair_matches.idA==idA) & (~(df_pair_matches.idB.isin(matched_idBs)))].sort_values(by='avg_match_score')[['idB','avg_match_score','area_diff']].values
            if idA_matches.shape[0] > 0:
                matched_idB, match_score, area_diff = idA_matches[0][0],idA_matches[0][1],idA_matches[0][2]
                matched_idBs.append(matched_idB)
                inverse_pairs[(matched_idB,idA)] = (match_score, area_diff)

        final_pairs = []
        paired_idAs, paired_idBs = [],[]
        for pair_key in direct_pairs:
            if pair_key in inverse_pairs:
                if (pair_key[0] not in paired_idAs) & (pair_key[0] not in paired_idBs):
                    final_pairs.append((pair_key, direct_pairs[pair_key],inverse_pairs[pair_key]))
                    paired_idAs.append(pair_key[0])
                    paired_idBs.append(pair_key[1])
            elif (direct_pairs[pair_key][0] < MATCH_PAIR_MAX_THRESHOLD) & (direct_pairs[pair_key][1] < MATCH_PAIR_AREA_THRESHOLD):
                if (pair_key[0] not in paired_idAs) & (pair_key[0] not in paired_idBs):
                    final_pairs.append((pair_key, direct_pairs[pair_key],-1))
                    paired_idAs.append(pair_key[0])
                    paired_idBs.append(pair_key[1])
        
        for pair_key in inverse_pairs:
            if (pair_key not in direct_pairs) & (inverse_pairs[pair_key][0] < MATCH_PAIR_MAX_THRESHOLD) & (inverse_pairs[pair_key][1] < MATCH_PAIR_AREA_THRESHOLD):
                if (pair_key[0] not in paired_idAs) & (pair_key[0] not in paired_idBs):
                    final_pairs.append((pair_key, -1,inverse_pairs[pair_key]))
                    paired_idAs.append(pair_key[0])
                    paired_idBs.append(pair_key[1])
        session_pair_matches[(session_keys[sessionA_idx], session_keys[sessionB_idx])] = final_pairs


        
        


In [ ]:
all_id_sets = []
for session_pair in session_pair_matches:
    id_pairs_info = session_pair_matches[session_pair]
    for (id_pair, direct_match_score, inv_match_score) in id_pairs_info:
        found_pair = False
        for id_set in all_id_sets:
            if (id_pair[0] in id_set) or (id_pair[1] in id_set):
                found_pair =  True
                id_set.add(id_pair[0])
                id_set.add(id_pair[1])
                break
        # print(all_id_sets, id_pair, found_pair)        
        if not found_pair:
            new_set = set()
            new_set.add(id_pair[0])
            new_set.add(id_pair[1])
            all_id_sets.append(new_set)


In [ ]:
id_pair = ('0424_5', '0501_11')


In [ ]:
def id_pair_in_gt(df_gt, id_pair):
    s1, id1 = id_pair[0].split("_")
    s2, id2 = id_pair[1].split("_")
    for gt_idx, gt_row in df_gt.iterrows():
        s1_matches = list(map(float,str(gt_row[f'Session-{s1}']).split(",")))
        s2_matches = list(map(float,str(gt_row[f'Session-{s2}']).split(",")))
        if (float(id1) in s1_matches) & (float(id2) in s2_matches):
            return True
    return False


In [ ]:
num_true_pairs, num_total_pairs = 0,0
predicted_pairs = []
for session_pair in session_pair_matches:
    id_pairs_info = session_pair_matches[session_pair]
    for (id_pair, direct_match_score, inv_match_score) in id_pairs_info:
        if session_cols.index(f'Session-{id_pair[0].split("_")[0]}') < session_cols.index(f'Session-{id_pair[1].split("_")[0]}'):
            predicted_pairs.append([id_pair[0],id_pair[1]])
        else:
            predicted_pairs.append([id_pair[1],id_pair[0]])
        if id_pair_in_gt(df_gt, id_pair):
            # print("True Pair", (id_pair, direct_match_score, inv_match_score))
            print(id_pair)
            num_true_pairs+=1
        else:
            ...
            # print("False Pair", (id_pair, direct_match_score, inv_match_score))
        num_total_pairs+=1
df_predicted = pd.DataFrame(predicted_pairs, columns=['idA','idB'])
df_predicted['pred']=1
# df_predicted


In [ ]:
df_true_matches[df_true_matches.idA=='0410_2']


In [ ]:
session_cols


In [ ]:
num_true_pairs, num_total_pairs


In [ ]:
df_all_matches = pd.merge(df_true_matches, df_predicted,on=['idA','idB'], how='outer')
df_all_matches.loc[df_all_matches['gt'].isnull(),'gt'] = 0
df_all_matches.loc[df_all_matches['pred'].isnull(),'pred'] = 0
df_all_matches


In [ ]:
df_all_matches.to_csv(f"case_studies/results/across_session_{type}_matches_{course}.csv", index=False)


In [ ]:
df_all_matches[(df_all_matches.pred==1) & (df_all_matches['gt']==1)].shape, df_all_matches.shape


In [ ]:
print(precision_score(df_all_matches['gt'], df_all_matches['pred'], average=None), recall_score(df_all_matches['gt'], df_all_matches['pred'], average=None))


In [ ]:
precision_score(df_all_matches['gt'], df_all_matches['pred']), recall_score(df_all_matches['gt'], df_all_matches['pred'])


# Hyperparameter optimization

In [ ]:
threshold_optimization = []
match_threshold_choices = np.arange(0.15,1,0.05)
match_pair_max_threshold_choices = np.arange(0,0.5,0.05)
match_pair_area_threshold_choices = np.arange(0,100,5)
hyperparameter_results =[] 

for (MATCH_THRESHOLD, MATCH_PAIR_MAX_THRESHOLD, MATCH_PAIR_AREA_THRESHOLD) in product(match_threshold_choices, match_pair_max_threshold_choices, match_pair_area_threshold_choices):
    if MATCH_THRESHOLD<MATCH_PAIR_MAX_THRESHOLD:
        continue
    final_matches = []
    for (sessionA, sessionB) in product(sorted(sessions), sorted(sessions)):
        sessionA_key, sessionB_key = sessionA.split("_")[-1][4:8], sessionB.split("_")[-1][4:8]
        if not (sessionA==sessionB):
            # if sessionA not in session_matches:
            # match session A and session B based on gaze clustering
            match_scores_gaze = {}
            match_scores_clu  = {}
            for idA,idB in product(course_input_dict[sessionA].keys(), course_input_dict[sessionB].keys()):
                # check if there exists a gt match for idA in SessionB
                # idA_key = f'{sessionA_key}_{idA}'
                # idB_key = f'{sessionB_key}_{idB}'
                # idA_matches = df_true_matches[(df_true_matches.sessionB==sessionB_key) & (df_true_matches.idA==idA_key)].idB.values
                # if len(idA_matches)<=0:
                #     idA_matches = df_true_matches[(df_true_matches.sessionA==sessionB_key) & (df_true_matches.idB==idA_key)].idA.values
                #     if len(idA_matches)<=0:
                #         # print(f"Skipping {idA},{idB} as no matches for {idA_key} in {sessionB_key}")
                #         continue
                # idB_matches = df_true_matches[(df_true_matches.sessionA==sessionA_key) & (df_true_matches.idB==idB_key)].idA.values
                # if len(idB_matches) <= 0:
                #     idB_matches = df_true_matches[(df_true_matches.sessionB==sessionA_key) & (df_true_matches.idA==idB_key)].idB.values
                #     if len(idB_matches) <= 0:
                #         # print(f"Skipping {idA},{idB} as no matches for {idB_key} in {sessionA_key}")
                #         continue

                gaze_embA, gaze_embB = course_input_dict[sessionA][idA]['gaze_emb'], course_input_dict[sessionB][idB]['gaze_emb']                
                clu_embA, clu_embB = course_input_dict[sessionA][idA]['cluster_emb'], course_input_dict[sessionB][idB]['cluster_emb']                
                
                if idA not in match_scores_gaze:
                    match_scores_gaze[idA] = {}
                if idA not in match_scores_clu:
                    match_scores_clu[idA] = {}
                    
                if gaze_embA is None or gaze_embB is None:
                    match_scores_gaze[idA][idB] = np.inf
                else:
                    match_distance = cdist(gaze_embA.reshape(1,-1), gaze_embB.reshape(1,-1))[0][0]
                    match_scores_gaze[idA][idB] = match_distance
    
                if clu_embA is None or clu_embB is None:
                    match_scores_clu[idA][idB] = np.inf
                else:
                    match_distance = cdist(clu_embA.reshape(1,-1), clu_embB.reshape(1,-1))[0][0]
                    match_scores_clu[idA][idB] = match_distance
            
            df_match_gaze = pd.DataFrame(match_scores_gaze) 
            df_match_clu = pd.DataFrame(match_scores_clu) 
            gaze_cols = df_match_gaze.columns.values.tolist()
            clu_cols = df_match_clu.columns.values.tolist()
            all_cols = np.unique(gaze_cols+clu_cols)
            for col in all_cols:
                if col not in df_match_clu.columns:
                    df_match_clu[col] = np.inf
                if col not in df_match_gaze.columns:
                    df_match_clu[col] = np.inf
                sessionB_matches = deepcopy(df_match_clu[col]).sort_values().head(3).index.values.tolist() + \
                                    deepcopy(df_match_gaze[col]).sort_values().head(3).index.values.tolist()
                sessionB_matches = np.unique(sessionB_matches)
                for match_id in sessionB_matches:
                    if (match_scores_clu[col][match_id]<MATCH_THRESHOLD) | (match_scores_gaze[col][match_id]<MATCH_THRESHOLD):
                        col_face_area = course_input_dict[sessionA][col]['face_width_med'] * course_input_dict[sessionA][col]['face_height_med']
                        match_face_area = course_input_dict[sessionB][match_id]['face_width_med'] * course_input_dict[sessionB][match_id]['face_height_med']
                        rel_diff = np.abs(col_face_area-match_face_area)*100/min(col_face_area,match_face_area)
                        final_matches.append((sessionA_key, sessionB_key, f'{sessionA_key}_{col}', f'{sessionB_key}_{match_id}', match_scores_gaze[col][match_id], match_scores_clu[col][match_id], col_face_area, match_face_area))
    
    df_final_matches = pd.DataFrame(final_matches, columns=['sessionA','sessionB','idA','idB','match_score_gaze','match_score_clu','face_areaA','face_areaB'])
    
    
    final_course_ids = {}
    session_pair_matches = {}
    session_keys = df_final_matches.sessionA.unique()
    
    for sessionA_idx in range(len(session_keys)):
        for sessionB_idx in range(sessionA_idx+1, len(session_keys)):
            # find all direct pairs
            
            direct_pairs = {}
            s1, s2 = session_keys[sessionA_idx], session_keys[sessionB_idx]
            df_pair_matches = df_final_matches[(df_final_matches.sessionA==s1) & (df_final_matches.sessionB==s2)]
            df_pair_matches = df_pair_matches.sort_values(by=['face_areaA','face_areaB'],ascending=False)
            df_pair_matches['area_diff'] = np.abs(df_pair_matches['face_areaA'] - df_pair_matches['face_areaB'])*100/np.minimum(df_pair_matches['face_areaA'],df_pair_matches['face_areaB'])
            df_pair_matches['avg_match_score'] = (df_pair_matches['match_score_gaze'] + df_pair_matches['match_score_clu'])/2
            # df_pair_matches['avg_match_score'] = df_pair_matches['match_score_gaze']
            # df_pair_matches['avg_match_score'] = df_pair_matches['match_score_clu']
            # get all eligible pairs with direct pairing
            matched_idBs = []
            for idA in df_pair_matches.idA.unique():
                idA_matches = df_pair_matches[(df_pair_matches.idA==idA) & (~(df_pair_matches.idB.isin(matched_idBs)))].sort_values(by='avg_match_score')[['idB','avg_match_score','area_diff']].values
                if idA_matches.shape[0] > 0:
                    matched_idB, match_score, area_diff = idA_matches[0][0],idA_matches[0][1],idA_matches[0][2]
                    matched_idBs.append(matched_idB)
                    direct_pairs[(idA,matched_idB)] = (match_score, area_diff)
    
            inverse_pairs = {}
            s1, s2 = session_keys[sessionB_idx], session_keys[sessionA_idx]
            df_pair_matches = df_final_matches[(df_final_matches.sessionA==s1) & (df_final_matches.sessionB==s2)]
            df_pair_matches = df_pair_matches.sort_values(by=['face_areaA','face_areaB'],ascending=False)
            df_pair_matches['area_diff'] = np.abs(df_pair_matches['face_areaA'] - df_pair_matches['face_areaB'])*100/np.minimum(df_pair_matches['face_areaA'],df_pair_matches['face_areaB'])
            df_pair_matches['avg_match_score'] = (df_pair_matches['match_score_gaze'] + df_pair_matches['match_score_clu'])/2
            # df_pair_matches['avg_match_score'] = df_pair_matches['match_score_gaze']
            # df_pair_matches['avg_match_score'] = df_pair_matches['match_score_clu']
            # get all eligible pairs with direct pairing
            matched_idBs = []
            for idA in df_pair_matches.idA.unique():
                idA_matches = df_pair_matches[(df_pair_matches.idA==idA) & (~(df_pair_matches.idB.isin(matched_idBs)))].sort_values(by='avg_match_score')[['idB','avg_match_score','area_diff']].values
                if idA_matches.shape[0] > 0:
                    matched_idB, match_score, area_diff = idA_matches[0][0],idA_matches[0][1],idA_matches[0][2]
                    matched_idBs.append(matched_idB)
                    inverse_pairs[(matched_idB,idA)] = (match_score, area_diff)
    
            final_pairs = []
            paired_idAs, paired_idBs = [],[]
            for pair_key in direct_pairs:
                if pair_key in inverse_pairs:
                    if (pair_key[0] not in paired_idAs) & (pair_key[0] not in paired_idBs):
                        final_pairs.append((pair_key, direct_pairs[pair_key],inverse_pairs[pair_key]))
                        paired_idAs.append(pair_key[0])
                        paired_idBs.append(pair_key[1])
                elif (direct_pairs[pair_key][0] < MATCH_PAIR_MAX_THRESHOLD) & (direct_pairs[pair_key][1] < MATCH_PAIR_AREA_THRESHOLD):
                    if (pair_key[0] not in paired_idAs) & (pair_key[0] not in paired_idBs):
                        final_pairs.append((pair_key, direct_pairs[pair_key],-1))
                        paired_idAs.append(pair_key[0])
                        paired_idBs.append(pair_key[1])
            
            for pair_key in inverse_pairs:
                if (pair_key not in direct_pairs) & (inverse_pairs[pair_key][0] < MATCH_PAIR_MAX_THRESHOLD) & (inverse_pairs[pair_key][1] < MATCH_PAIR_AREA_THRESHOLD):
                    if (pair_key[0] not in paired_idAs) & (pair_key[0] not in paired_idBs):
                        final_pairs.append((pair_key, -1,inverse_pairs[pair_key]))
                        paired_idAs.append(pair_key[0])
                        paired_idBs.append(pair_key[1])
            session_pair_matches[(session_keys[sessionA_idx], session_keys[sessionB_idx])] = final_pairs
    
    all_id_sets = []
    for session_pair in session_pair_matches:
        id_pairs_info = session_pair_matches[session_pair]
        for (id_pair, direct_match_score, inv_match_score) in id_pairs_info:
            found_pair = False
            for id_set in all_id_sets:
                if (id_pair[0] in id_set) or (id_pair[1] in id_set):
                    found_pair =  True
                    id_set.add(id_pair[0])
                    id_set.add(id_pair[1])
                    break
            # print(all_id_sets, id_pair, found_pair)        
            if not found_pair:
                new_set = set()
                new_set.add(id_pair[0])
                new_set.add(id_pair[1])
                all_id_sets.append(new_set)
    
    num_true_pairs, num_total_pairs = 0,0
    for session_pair in session_pair_matches:
        id_pairs_info = session_pair_matches[session_pair]
        for (id_pair, direct_match_score, inv_match_score) in id_pairs_info:
            if id_pair_in_gt(df_gt, id_pair):
                # print("True Pair", (id_pair, direct_match_score, inv_match_score))
                num_true_pairs+=1
            # else:
            num_total_pairs+=1
                # print("False Pair", (id_pair, direct_match_score, inv_match_score))
                
            
            
    if num_total_pairs > 0:
        hyperparameter_results.append([MATCH_THRESHOLD, MATCH_PAIR_MAX_THRESHOLD, MATCH_PAIR_AREA_THRESHOLD, num_true_pairs, num_total_pairs, num_true_pairs/num_total_pairs])
        print(MATCH_THRESHOLD, MATCH_PAIR_MAX_THRESHOLD, MATCH_PAIR_AREA_THRESHOLD, num_true_pairs, num_total_pairs, num_true_pairs/num_total_pairs)
    else:
        print(MATCH_THRESHOLD, MATCH_PAIR_MAX_THRESHOLD, MATCH_PAIR_AREA_THRESHOLD, num_true_pairs, num_total_pairs)


In [ ]:
df_results = pd.DataFrame(hyperparameter_results, columns=['max_thr','max_pair_thr','max_area_thr','num_true_pairs','num_total_pairs','precision'])
df_results.shape


In [ ]:
df_results[df_results.num_total_pairs>=25].sort_values(by='precision',ascending=False).head(20)


In [ ]:
# s1,s2 = '0228','0205'
# df_pair_matches = df_final_matches[(df_final_matches.sessionA==s1) & (df_final_matches.sessionB==s2)]
# df_pair_matches


In [ ]:
# df_pair_matches = df_pair_matches.sort_values(by=['face_areaA','face_areaB'],ascending=False)
# df_pair_matches['area_diff'] = np.abs(df_pair_matches['face_areaA'] - df_pair_matches['face_areaB'])*100/np.minimum(df_pair_matches['face_areaA'],df_pair_matches['face_areaB'])
# df_pair_matches['avg_match_score'] = (df_pair_matches['match_score_gaze'] + df_pair_matches['match_score_clu'])/2
# df_pair_matches[['idA','idB','area_diff','avg_match_score']]
# # df_pair_matches.sort_values(by='idA')


In [ ]:
# df_pair_matches.idA.unique()


In [ ]:
# get final pairs based on df_pair_matches
# final_pairs = {}
# matched_idBs = []
# pair_match_max=0.2
# for idA in df_pair_matches.idA.unique():
#     idA_matches = df_pair_matches[(df_pair_matches.idA==idA) & (~(df_pair_matches.idB.isin(matched_idBs)))].sort_values(by='avg_match_score')[['idB','avg_match_score']].values
#     if idA_matches.shape[0] > 0:
#         matched_idB, match_score = idA_matches[0][0],idA_matches[0][1]
#         if match_score<pair_match_max:
#             matched_idBs.append(matched_idB)
#             print(idA,matched_idB,match_score)  
#         else:
#             #potential pair
#             print("Potential Pair:", idA,matched_idB,match_score)  


In [ ]:
# get final pairs based on df_pair_matches
# final_pairs = {}
# matched_idBs = []
# # pair_match_max=0.2
# for idA in df_pair_matches.idA.unique():
#     idA_matches = df_pair_matches[(df_pair_matches.idA==idA) & (~(df_pair_matches.idB.isin(matched_idBs)))].sort_values(by='avg_match_score')[['idB','avg_match_score']].values
#     if idA_matches.shape[0] > 0:
#         matched_idB, match_score = idA_matches[0][0],idA_matches[0][1]
#         matched_idBs.append(matched_idB)
#         print(idA,matched_idB,match_score)


In [ ]:
df_pair_matches[(df_pair_matches.match_score_gaze<0.2) & (df_pair_matches.match_score_clu<0.2) & (df_pair_matches.area_diff<20) & (np.minimum(df_pair_matches.face_areaA, df_pair_matches.face_areaB)>np.median(df_pair_matches.face_areaA))]


In [ ]:
id_matches = {}
matched_idA, matched_idB = [],[]

best_match_1 = df_pair_matches[(df_pair_matches.match_score_gaze<0.2) & 
                                (df_pair_matches.match_score_clu<0.2) & 
                                (df_pair_matches.area_diff<20) & 
                                (np.minimum(df_pair_matches.face_areaA, df_pair_matches.face_areaB)>np.median(df_pair_matches.face_areaA))
                                ].sort_values(by='match_score_clu')
sprint(best_match_1)
for idx,row in best_match_1.iterrows():
    if row['idA'] in matched_idA:
        continue
    elif row['idB'] in matched_idB:
        continue
    else:
        id_matches[row['idA']] = row['idB']
        matched_idA.append(row['idA'])
        matched_idB.append(row['idB'])
df_next_matches = df_pair_matches[(~df_pair_matches.idA.isin(matched_idA)) & (~df_pair_matches.idB.isin(matched_idB))]
id_matches


In [ ]:
df_next_matches = pd.merge(df_next_matches, df_next_matches.groupby('idA',as_index=False)['idB'].count(), on='idA',suffixes=('','_count'))
df_next_matches


In [ ]:
df_single_matches = df_next_matches[df_next_matches.idB_count<=1]
best_match_2 = df_single_matches[(df_single_matches.match_score_gaze<0.25) & 
                                (df_single_matches.match_score_clu<0.25) & 
                                # (df_single_matches.area_diff<20) & 
                                (np.minimum(df_single_matches.face_areaA, df_single_matches.face_areaB)>np.median(df_pair_matches.face_areaA))
                                ].sort_values(by='match_score_gaze')
sprint(best_match_2)
for idx,row in best_match_2.iterrows():
    if row['idA'] in matched_idA:
        continue
    elif row['idB'] in matched_idB:
        continue
    else:
        id_matches[row['idA']] = row['idB']
        matched_idA.append(row['idA'])
        matched_idB.append(row['idB'])
df_next_matches = df_pair_matches[(~df_pair_matches.idA.isin(matched_idA)) & (~df_pair_matches.idB.isin(matched_idB))]
id_matches


In [ ]:
df_next_matches = pd.merge(df_next_matches, df_next_matches.groupby('idB',as_index=False)['idA'].count(), on='idB',suffixes=('','_count'))
df_next_matches


In [ ]:
best_match_3 = df_next_matches[(df_next_matches.idA_count<=1) & 
                                (df_next_matches.area_diff<20)].sort_values(by='match_score_gaze')
sprint(best_match_3)
for idx,row in best_match_3.iterrows():
    if row['idA'] in matched_idA:
        continue
    elif row['idB'] in matched_idB:
        continue
    else:
        id_matches[row['idA']] = row['idB']
        matched_idA.append(row['idA'])
        matched_idB.append(row['idB'])


In [ ]:
df_next_matches = df_pair_matches[(~df_pair_matches.idA.isin(matched_idA)) & (~df_pair_matches.idB.isin(matched_idB))]
id_matches


In [ ]:
df_next_matches = pd.merge(df_next_matches, df_next_matches.groupby('idB',as_index=False)['idA'].count(), on='idB',suffixes=('','_count'))
df_next_matches = pd.merge(df_next_matches, df_next_matches.groupby('idA',as_index=False)['idB'].count(), on='idA',suffixes=('','_count'))
df_next_matches.sort_values(by='idA_count')


In [ ]:
df_next_matches = pd.merge(df_next_matches, df_next_matches.groupby('idA',as_index=False).agg({'match_score_clu':lambda x: 1.0 if (len(x)<2) else sorted(x)[1]-sorted(x)[0]}), on='idA',suffixes=('','_best_diff'))
df_next_matches


In [ ]:
df_next_matches = pd.merge(df_next_matches, df_next_matches.groupby('idA',as_index=False).agg({'match_score_clu':lambda x: sorted(x)[0]}), on='idA',suffixes=('','_min_match'))
df_next_matches


In [ ]:
best_match_4 = df_next_matches[(df_next_matches.match_score_gaze<0.25) &
                (df_next_matches.match_score_clu_best_diff>0.05) &
                (df_next_matches.match_score_clu==df_next_matches.match_score_clu_min_match)].sort_values(by='match_score_clu')
sprint(best_match_4)
for idx,row in best_match_4.iterrows():
    if row['idA'] in matched_idA:
        continue
    elif row['idB'] in matched_idB:
        continue
    else:
        id_matches[row['idA']] = row['idB']
        matched_idA.append(row['idA'])
        matched_idB.append(row['idB'])
        


In [ ]:
df_next_matches = df_pair_matches[(~df_pair_matches.idA.isin(matched_idA)) & (~df_pair_matches.idB.isin(matched_idB))]
id_matches


In [ ]:
df_next_matches = df_next_matches.sort_values(by=['face_areaA'],ascending=False)
df_next_matches
